In [6]:
import pyspark
from pyspark.sql import SparkSession

In [7]:
import os
os.environ['PYSPARK_PYTHON'] = "C:/users/Administrator/AppData/Local/Programs/Python/Python311/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/Adminstrator/AppData/Local/Programs/Python/Python311/python.exe"

In [8]:
from pyspark.sql.functions import *
filepath = "D:\sf-fire-calls.csv"

In [9]:
def create_sparkSession():
    spark = SparkSession.builder.appName('fireanalysis').getOrCreate()
    return spark


In [10]:
def create_dataframe(spark, filepath):
    df = spark.read.csv(filepath, header=True, inferSchema=True)
    df1 = df.select('CallType','CallDate','City','Zipcode','Neighborhood','Delay')
  
    return df1
    

In [11]:
spark = create_sparkSession()
df = create_dataframe(spark,filepath)
df.printSchema()
    

root
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Delay: double (nullable = true)



In [12]:
def clean_dataset(df):
    df1 = df.withColumn('Date',to_date(col('CallDate'), 'MM/dd/yyyy')).drop('CallDate')
    df2 = df1.withColumn('Year',year(col('Date')))\
    .withColumn('Month',month(col('Date')))\
    .withColumn('Week',weekofyear(col('Date')))
    return df2

In [13]:
spark =create_sparkSession()
df = create_dataframe(spark,filepath)
df = clean_dataset(df)
df.show()


+----------------+----+-------+--------------------+---------+----------+----+-----+----+
|        CallType|City|Zipcode|        Neighborhood|    Delay|      Date|Year|Month|Week|
+----------------+----+-------+--------------------+---------+----------+----+-----+----+
|  Structure Fire|  SF|  94109|     Pacific Heights|     2.95|2002-01-11|2002|    1|   2|
|Medical Incident|  SF|  94124|Bayview Hunters P...|      4.7|2002-01-11|2002|    1|   2|
|Medical Incident|  SF|  94102|          Tenderloin|2.4333334|2002-01-11|2002|    1|   2|
|    Vehicle Fire|  SF|  94110|      Bernal Heights|      1.5|2002-01-11|2002|    1|   2|
|          Alarms|  SF|  94109|    Western Addition|3.4833333|2002-01-11|2002|    1|   2|
|  Structure Fire|  SF|  94105|Financial Distric...|     1.75|2002-01-11|2002|    1|   2|
|          Alarms|  SF|  94112|Oceanview/Merced/...|2.7166667|2002-01-11|2002|    1|   2|
|          Alarms|  SF|  94102|          Tenderloin|1.7833333|2002-01-11|2002|    1|   2|
|Medical I

In [14]:
def clean_data(df):
    df1 = df.withColumn('Date',to_date(col('callDate'),'MM/dd/yyyy')).drop('callDate')
    df2 = df1.withColumn('Year',year(col('Date')))\
    .withColumn('Month',month(col('Month')))\
    .withColumn('Week',weekofyear(col('Week')))

In [15]:
def mapSeason(data):
    if 2<data<6:
        return 'Spring'
    elif 5 < data < 9:
        return 'Summer'
    elif 8 < data < 12:
        return 'Autumn'
    else:
        return 'Winter'
seasonUDF = udf(mapSeason,StringType())
clean_df = df.withColumn('Season',seasonUDF(col('Month')))

In [16]:
clean_df.show()

+----------------+----+-------+--------------------+---------+----------+----+-----+----+------+
|        CallType|City|Zipcode|        Neighborhood|    Delay|      Date|Year|Month|Week|Season|
+----------------+----+-------+--------------------+---------+----------+----+-----+----+------+
|  Structure Fire|  SF|  94109|     Pacific Heights|     2.95|2002-01-11|2002|    1|   2|Winter|
|Medical Incident|  SF|  94124|Bayview Hunters P...|      4.7|2002-01-11|2002|    1|   2|Winter|
|Medical Incident|  SF|  94102|          Tenderloin|2.4333334|2002-01-11|2002|    1|   2|Winter|
|    Vehicle Fire|  SF|  94110|      Bernal Heights|      1.5|2002-01-11|2002|    1|   2|Winter|
|          Alarms|  SF|  94109|    Western Addition|3.4833333|2002-01-11|2002|    1|   2|Winter|
|  Structure Fire|  SF|  94105|Financial Distric...|     1.75|2002-01-11|2002|    1|   2|Winter|
|          Alarms|  SF|  94112|Oceanview/Merced/...|2.7166667|2002-01-11|2002|    1|   2|Winter|
|          Alarms|  SF|  94102

In [17]:
import calendar

def month_num_to_name(month_num):
    if 1 <= month_num <= 12:
        return calendar.month_name[month_num]
    else:
        return None 

In [18]:
yearly_fire_call = clean_df.select('Year').groupBy('Year').count().orderBy('Year')
yearly_fire_call.show()

+----+-----+
|Year|count|
+----+-----+
|2000| 5459|
|2001| 7713|
|2002| 8090|
|2003| 8499|
|2004| 8283|
|2005| 8282|
|2006| 8174|
|2007| 8255|
|2008| 8869|
|2009| 8789|
|2010| 9341|
|2011| 9735|
|2012| 9674|
|2013|10020|
|2014|10775|
|2015|11458|
|2016|11609|
|2017|12135|
|2018|10136|
+----+-----+



In [19]:
diff_callsin_2018 = clean_df.select('CallType').where(col('Year')==2018).distinct().show(truncate=True)
#diff_callsin_2018.show()

+--------------------+
|            CallType|
+--------------------+
|Elevator / Escala...|
|              Alarms|
|Odor (Strange / U...|
|Citizen Assist / ...|
|              HazMat|
|        Vehicle Fire|
|               Other|
|        Outside Fire|
|   Traffic Collision|
|       Assist Police|
|Gas Leak (Natural...|
|        Water Rescue|
|   Electrical Hazard|
|      Structure Fire|
|    Medical Incident|
|          Fuel Spill|
|Smoke Investigati...|
|Train / Rail Inci...|
|           Explosion|
|  Suspicious Package|
+--------------------+



In [20]:
clean_df.createOrReplaceTempView('Fire')
spark.sql('select Year from Fire where Delay > 2.00').show()

+----+
|Year|
+----+
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
|2002|
+----+
only showing top 20 rows



In [21]:
#1
clean_df.select('Year').groupBy('Year').count().orderBy('Year', ascending=True).show()
 

+----+-----+
|Year|count|
+----+-----+
|2000| 5459|
|2001| 7713|
|2002| 8090|
|2003| 8499|
|2004| 8283|
|2005| 8282|
|2006| 8174|
|2007| 8255|
|2008| 8869|
|2009| 8789|
|2010| 9341|
|2011| 9735|
|2012| 9674|
|2013|10020|
|2014|10775|
|2015|11458|
|2016|11609|
|2017|12135|
|2018|10136|
+----+-----+



In [22]:
clean_df.select('CallType').where(col('Year')== 2018).distinct().show(truncate=False)

+-------------------------------+
|CallType                       |
+-------------------------------+
|Elevator / Escalator Rescue    |
|Alarms                         |
|Odor (Strange / Unknown)       |
|Citizen Assist / Service Call  |
|HazMat                         |
|Vehicle Fire                   |
|Other                          |
|Outside Fire                   |
|Traffic Collision              |
|Assist Police                  |
|Gas Leak (Natural and LP Gases)|
|Water Rescue                   |
|Electrical Hazard              |
|Structure Fire                 |
|Medical Incident               |
|Fuel Spill                     |
|Smoke Investigation (Outside)  |
|Train / Rail Incident          |
|Explosion                      |
|Suspicious Package             |
+-------------------------------+



In [23]:
#3. Which week in the year 2018 had the most fire calls?
clean_df.select('Week').where(col('Year')==2018).groupBy('Week').count().orderBy('count', ascending=False).collect()[0][1]

259

In [24]:
max_month = clean_df.select('Week').where(col('Year') == 2018).groupBy('Week').count()
max_month.select('Week','count').filter(col('count') == max_month.agg({'count':'max'}).collect()[0][0]).collect()[0][0]

22

In [25]:
#4 Get monthly count of fire calls based on year 
clean_df.groupBy('Month', 'Year').count().orderBy('Year', 'Month').show()

+-----+----+-----+
|Month|Year|count|
+-----+----+-----+
|    4|2000|  335|
|    5|2000|  680|
|    6|2000|  585|
|    7|2000|  668|
|    8|2000|  678|
|    9|2000|  655|
|   10|2000|  620|
|   11|2000|  595|
|   12|2000|  643|
|    1|2001|  622|
|    2|2001|  613|
|    3|2001|  692|
|    4|2001|  636|
|    5|2001|  682|
|    6|2001|  672|
|    7|2001|  646|
|    8|2001|  660|
|    9|2001|  577|
|   10|2001|  673|
|   11|2001|  619|
+-----+----+-----+
only showing top 20 rows



In [26]:
#5 Give monthly report of fire call types for selected year
year_inp = input("Enter the year")
clean_df.filter(col('Year') == year_inp).groupBy('CallType','Month').count().orderBy('CallType','Month').show()

Enter the year 2018


+--------------------+-----+-----+
|            CallType|Month|count|
+--------------------+-----+-----+
|              Alarms|    1|  122|
|              Alarms|    2|  102|
|              Alarms|    3|  133|
|              Alarms|    4|   93|
|              Alarms|    5|  107|
|              Alarms|    6|   88|
|              Alarms|    7|  117|
|              Alarms|    8|  131|
|              Alarms|    9|   97|
|              Alarms|   10|  130|
|              Alarms|   11|   24|
|       Assist Police|    2|    1|
|Citizen Assist / ...|    1|   15|
|Citizen Assist / ...|    2|   12|
|Citizen Assist / ...|    3|    8|
|Citizen Assist / ...|    4|   11|
|Citizen Assist / ...|    5|    7|
|Citizen Assist / ...|    6|    5|
|Citizen Assist / ...|    7|    9|
|Citizen Assist / ...|    8|   11|
+--------------------+-----+-----+
only showing top 20 rows



In [27]:
from pyspark.sql.window import Window
#6 Give top 5 fire call types for every season of selected year
season_count = clean_df.select('CallType', 'Season').filter(col('Year') == 2014).groupBy('Season', 'CallType').count()\
    .withColumn('Rank', dense_rank().over(Window.partitionBy('Season').orderBy(col('count').desc())))\
        .orderBy('Season', 'Rank', ascending = [1,1]).filter(col('Rank')<6) # [1,1] means that ascending is true for both first and second column
season_count.show()

+------+--------------------+-----+----+
|Season|            CallType|count|Rank|
+------+--------------------+-----+----+
|Autumn|    Medical Incident| 1832|   1|
|Autumn|      Structure Fire|  319|   2|
|Autumn|              Alarms|  314|   3|
|Autumn|   Traffic Collision|  120|   4|
|Autumn|Citizen Assist / ...|   26|   5|
|Spring|    Medical Incident| 1801|   1|
|Spring|      Structure Fire|  314|   2|
|Spring|              Alarms|  281|   3|
|Spring|   Traffic Collision|  112|   4|
|Spring|Citizen Assist / ...|   36|   5|
|Summer|    Medical Incident| 1770|   1|
|Summer|      Structure Fire|  322|   2|
|Summer|              Alarms|  269|   3|
|Summer|   Traffic Collision|  112|   4|
|Summer|        Outside Fire|   37|   5|
|Winter|    Medical Incident| 1773|   1|
|Winter|              Alarms|  337|   2|
|Winter|      Structure Fire|  310|   3|
|Winter|   Traffic Collision|  111|   4|
|Winter|Citizen Assist / ...|   36|   5|
+------+--------------------+-----+----+



In [28]:
#7 Whether fire type calls are seasonal? 
clean_df.groupBy("Season", "CallType").count().orderBy("CallType", "Season").show(1000, truncate = False)
#based on the results, few call types like explosion, Gas Leak, Outside fire are seasonal. 

+------+--------------------------------------------+-----+
|Season|CallType                                    |count|
+------+--------------------------------------------+-----+
|Autumn|Administrative                              |1    |
|Spring|Administrative                              |1    |
|Winter|Administrative                              |1    |
|Autumn|Aircraft Emergency                          |10   |
|Spring|Aircraft Emergency                          |6    |
|Summer|Aircraft Emergency                          |10   |
|Winter|Aircraft Emergency                          |10   |
|Autumn|Alarms                                      |4980 |
|Spring|Alarms                                      |4641 |
|Summer|Alarms                                      |4910 |
|Winter|Alarms                                      |4875 |
|Autumn|Assist Police                               |6    |
|Spring|Assist Police                               |9    |
|Summer|Assist Police                   

In [29]:
#8 What months within the year 2018 saw the highest number of fire calls?
highest_month = clean_df.filter(col('Year')==2018).groupby('Month').count().orderBy(col('count'), ascending = False)
print("Month {} has the highest number of calls in 2018, with {} calls. ".format(month_num_to_name(highest_month.collect()[0][0]), highest_month.collect()[0][1]))

Month October has the highest number of calls in 2018, with 1068 calls. 


In [30]:
#9. Find which type of fire call is major calltype in each year 
from pyspark.sql.window import Window
win = Window.partitionBy('Year').orderBy(col('count').desc())
clean_df.groupBy('Year', 'CallType').count().withColumn('Rank', dense_rank().over(win)).filter(col('Rank')<2).show()

+----+----------------+-----+----+
|Year|        CallType|count|Rank|
+----+----------------+-----+----+
|2000|Medical Incident| 3408|   1|
|2001|Medical Incident| 4653|   1|
|2002|Medical Incident| 5046|   1|
|2003|Medical Incident| 5056|   1|
|2004|Medical Incident| 5137|   1|
|2005|Medical Incident| 5084|   1|
|2006|Medical Incident| 5027|   1|
|2007|Medical Incident| 5114|   1|
|2008|Medical Incident| 5692|   1|
|2009|Medical Incident| 5671|   1|
|2010|Medical Incident| 6186|   1|
|2011|Medical Incident| 6413|   1|
|2012|Medical Incident| 6296|   1|
|2013|Medical Incident| 6690|   1|
|2014|Medical Incident| 7176|   1|
|2015|Medical Incident| 7812|   1|
|2016|Medical Incident| 7999|   1|
|2017|Medical Incident| 8330|   1|
|2018|Medical Incident| 7004|   1|
+----+----------------+-----+----+



In [33]:
#10 Find out average delay in response for each call type 
avg_delay = clean_df.groupBy('CallType').agg(avg('Delay').alias('Avg Delay'))
avg_delay.show()

+--------------------+------------------+
|            CallType|         Avg Delay|
+--------------------+------------------+
|Elevator / Escala...| 4.337821933487859|
|  Aircraft Emergency|3.7731481500000004|
|              Alarms| 3.542729054508399|
|Odor (Strange / U...|       4.947959182|
|Citizen Assist / ...| 5.473342576604596|
|              HazMat| 7.527016126612904|
|           Explosion| 4.110674168539325|
|           Oil Spill| 4.977777761904762|
|        Vehicle Fire| 3.903922713407494|
|  Suspicious Package|        6.57666672|
|Extrication / Ent...| 4.391666678571428|
|               Other| 5.505155432421977|
|        Outside Fire| 4.181948425367717|
|   Traffic Collision|3.7891320888732363|
|       Assist Police|26.981903994285716|
|Gas Leak (Natural...| 4.583398778403141|
|        Water Rescue| 5.507748342145695|
|   Electrical Hazard| 5.178112038174275|
|   High Angle Rescue| 6.048958375000001|
|      Structure Fire| 3.679561015471934|
+--------------------+------------

In [34]:
#11 Find which calltype has maximum average delay time. 
avg_delay.agg({'Avg Delay': 'max'}).collect()
print("{} has the highest avg delay time of {}.".format(avg_delay.collect()[0][0], avg_delay.collect()[0][1]))

Elevator / Escalator Rescue has the highest avg delay time of 4.337821933487859.


In [48]:
#12 Which neighborhood in San Francisco generated the most fire calls in 2018? 
clean_df.filter((col('Year')==2018) & ((col('City') == 'SF') | (col('City') == 'San Francisco'))).groupBy('Neighborhood').count().orderBy('count', ascending = False).show(1)

+------------+-----+
|Neighborhood|count|
+------------+-----+
|  Tenderloin| 1393|
+------------+-----+
only showing top 1 row



In [37]:
#13 Which neighborhoods had the worst response times to fire calls in 2018? 
Delay_neigh = clean_df.filter(col('Year') == 2018).groupBy('Neighborhood', 'Delay').count().orderBy(col('Delay'), ascending = False)
print(Delay_neigh.collect()[0][0], Delay_neigh.collect()[0][1])


Chinatown 491.26666


In [62]:
!pip install pandas>=1.0.5 matplotlib


[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
#14  Find out calltype whose average response delay time is maximum, increases, decreases or has no relation over years. 
Averaged_data = clean_df.groupBy('CallType','Year').agg(avg('Delay').alias('Avg Delay')).orderBy(col('CallType'), col('Year'))
# print("Maximum Avergae Delay {} for calltype '{}'".format(Averaged_data.orderBy(col('Avg Delay'), ascending = False).collect()[0][2], Averaged_data.orderBy(col('Avg Delay'), ascending = False).collect()[0][0]))
Averaged_data.show(1000)

# import matplotlib.pyplot as plt
# avg_delay_pd = Averaged_data.toPandas()
# plt.figure(figsize=(12,5))
# plt.plot(avg_delay_pd['Year'], avg_delay_pd['Avg Delay'], marker='o', color='teal', linewidth=2)
# plt.title('Average Response Delay by Year')
# plt.xlabel('Year')
# plt.ylabel('Average Delay (minutes)')
# plt.grid(True)
# plt.show()


+--------------------+----+------------------+
|            CallType|Year|         Avg Delay|
+--------------------+----+------------------+
|      Administrative|2005|         31.983334|
|      Administrative|2006|               1.8|
|      Administrative|2017|               3.0|
|  Aircraft Emergency|2000| 3.905555533333333|
|  Aircraft Emergency|2001|       2.616666675|
|  Aircraft Emergency|2002|        4.14666662|
|  Aircraft Emergency|2003|         13.166667|
|  Aircraft Emergency|2004|         2.5916667|
|  Aircraft Emergency|2005|        4.29166675|
|  Aircraft Emergency|2006|3.2111111166666664|
|  Aircraft Emergency|2007| 3.094444333333333|
|  Aircraft Emergency|2009|         3.0083335|
|  Aircraft Emergency|2011|3.5944443333333336|
|  Aircraft Emergency|2012|        4.65833335|
|  Aircraft Emergency|2013|         2.4333334|
|  Aircraft Emergency|2014|              7.75|
|  Aircraft Emergency|2015|         1.1333333|
|              Alarms|2000|3.0111468393086813|
|            

In [83]:
 #15 For each year find out which city has more calltypes
clean_df.groupBy('Year', 'City', 'CallType').count()\
    .withColumn('Rank', dense_rank()\
        .over(Window.partitionBy(col('Year')).orderBy(col('count').desc()))).filter(col('Rank') == 1).show()

+----+-------------+----------------+-----+----+
|Year|         City|        CallType|count|Rank|
+----+-------------+----------------+-----+----+
|2000|           SF|Medical Incident| 3399|   1|
|2001|           SF|Medical Incident| 4619|   1|
|2002|           SF|Medical Incident| 5022|   1|
|2003|           SF|Medical Incident| 5026|   1|
|2004|           SF|Medical Incident| 5104|   1|
|2005|           SF|Medical Incident| 5063|   1|
|2006|           SF|Medical Incident| 4997|   1|
|2007|           SF|Medical Incident| 5073|   1|
|2008|           SF|Medical Incident| 5666|   1|
|2009|           SF|Medical Incident| 5638|   1|
|2010|           SF|Medical Incident| 6141|   1|
|2011|           SF|Medical Incident| 6361|   1|
|2012|           SF|Medical Incident| 6249|   1|
|2013|           SF|Medical Incident| 6626|   1|
|2014|San Francisco|Medical Incident| 4682|   1|
|2015|San Francisco|Medical Incident| 7733|   1|
|2016|San Francisco|Medical Incident| 7914|   1|
|2017|San Francisco|

In [55]:
#16  For every year find count of calltypes  for 5 cities which has more calls.
win = Window.partitionBy(col('Year')).orderBy(col('count').desc())
clean_df.groupBy('Year', 'CallType', 'City').count().withColumn('Rank', dense_rank().over(win)).filter(col('Rank') <6).show(1000)

+----+--------------------+-------------+-----+----+
|Year|            CallType|         City|count|Rank|
+----+--------------------+-------------+-----+----+
|2000|    Medical Incident|           SF| 3399|   1|
|2000|      Structure Fire|           SF| 1002|   2|
|2000|              Alarms|           SF|  620|   3|
|2000|Citizen Assist / ...|           SF|  124|   4|
|2000|               Other|           SF|   99|   5|
|2001|    Medical Incident|           SF| 4619|   1|
|2001|      Structure Fire|           SF| 1542|   2|
|2001|              Alarms|           SF|  868|   3|
|2001|               Other|           SF|  189|   4|
|2001|Citizen Assist / ...|           SF|  122|   5|
|2002|    Medical Incident|           SF| 5022|   1|
|2002|      Structure Fire|           SF| 1571|   2|
|2002|              Alarms|           SF|  827|   3|
|2002|               Other|           SF|  171|   4|
|2002|Citizen Assist / ...|           SF|  140|   5|
|2003|    Medical Incident|           SF| 5026

In [99]:
 #17 Is there a correlation between neighborhood, zip code, and number of fire calls?
clean_df.groupBy('Neighborhood','Zipcode').count().stat.corr('Zipcode','count')
#lower value of the correlation function suggests that there is no coorelation between the neighborhood, zipcode and the number of fire calls

0.06771510346381833